# ECH 60 Spring 2020 week 8
&copy; Harishankar Manikantan, Chemical Engineering, UC Davis 

**[(8.1) Integrals as Finite Sums](#sums) (Monday, Wednesday)**
*  [Rectangle methods or Riemann sums](#rectangle) <br>
*  [Taylor series expansions and truncation errors](#taylor) <br>
*  [Trapezoidal rule](#trapz) <br>
*  [Simpson's rule ](#simpson)

**[(8.2) More Integration Strategies](#more) (Friday)**
*  [Multidimensional integrals](#multi) <br>
*  [Python inbuilt routines](#inbuilt) <br>

**[Practice problems](#exer)**



<a id='sums'></a>



## (8.1) Integrals as Finite Sums

Integration gives a cumulative measure of the data or function. With time dependent functions (like velocity), integration gives the sum of small bits of the function added up at every step of the independent variable (so adding up velocity times time at every instant gives total distance travelled). With spatially-varying functions (like concentration or density, ...), integration gives the total valiue over a specified range (total number of molecules, total mass etc, ...). Geometrically, integration provides the area under a curve. In analogy with finite differences to approximate derivatives, we can think of finite 'sums' to add up bits at every point to get the total area or the total integral.

We know how to analytically obtain integrals of simple functions. But, much like differentiation, some functions are too complicated (or even impossible) to integrate by hand and require a numerical approach to approximate. Similarly, discrete measurement or data points can be integrated numerically using finite sums without knowing the functional relationship between the dependent and independent variables.

<a id='rectangle'></a>



###  Rectangle methods or Riemann sums

Given the function $y(x)=x^3$, can we *numerically* find the integral between $x=2$ and $x=4$? We know that 

$$ \int_2^4 x^3 \,dx = \left. \frac{x^4}{4}\, \right|_{x=2}^{x=4} = \frac{4^4-2^4}{4} = 60. $$

However, as is often the case with real measurements and data sets or complicated functions, let's say we did not know the exact integral formula. What could we do?

First, think of relationship between the function $y(x)$ and its indefinite integral $Y(x)$: by the fundamental theorem of calculus, $y(x)$ is the derivative of $Y(x)$ and so

$$ y(x)=\frac{dY}{dx}  = \lim_{h \to 0} \frac{Y(x+h)-Y(x)}{h} $$

This means that if we choose a small enough gap size $h$, we can rearrange to find the *definite* integral between $Y(x)$ and $Y(x+h)$ as 

$$ \int_x^{x+h} y \,dx=Y(x+h) - Y(x) = h \, y(x).$$

Of course, this only works if $h\rightarrow 0$. What if we wanted to integrate across a bigger range like $x\in[a,b]$, like in the example above from $x=2$ to $x=3$?

The simplest way to do this is by breaking up the range $[a,b]$ into small chunks $[a,a+h], [a+h,a+2h], \ldots$, each of width $h$, and performing the above operation for each bit, and to add them all up:

$$ Y(b) - Y(a) = \left[ Y(a+h) - Y(a) \right] + \left[ Y(a+2h) - Y(a+h) \right] + \left[ Y(a+3h) - Y(a+2h) \right] + \ldots + \left[ Y(b) - Y(b-h) \right] $$

And each of these tiny chunks (becasue $h$ is small) can be done using the previous step, to give

$$ Y(b) - Y(a)= h \,y(a) + h\,y(a+h) + h\,y(a+2h) + \ldots + h\,y(b-h)   $$

In other words, the definite integral is simply the sum of the functional value evaluated at intermediate points (including the lower limit $a$ but excluding the upper limit $b$), each weighted by $h$. This is the exact analog of the forward difference formula for derivatives: each 'slice' is now a rectangle with the functional value at the current point as the height and the distance to the forward point as the width. This approximation to integration is called the *left rectangle method* or the *left Riemann sum*.

And clearly, the results is more accurate if we choose a smaller $h$. Let's try this:

In [23]:
import numpy as np

# function
y=lambda x: x**3

# integral boundaries
a=2
b=4

# h is difference between current and next time
h=0.1

# create an array of intermediate points with spacing h
# include a, exclude b
x=np.arange(a,b,h)

# perform sum all bits
Y=sum(h*y(x))

print('Approximate integral with a gap size of %.4f is %.4f' %(h,Y))

Approximate integral with a gap size of 0.1000 is 57.2300


Try smaller values of $h$ to get a better approximate (remember, we know the exact integral to be $60$ in this case).

And, much like finite differences, we are not restricted to 'forward' pieces. We can equally well create 'backward' or 'central' rectangles. For backward rectangles, the very first slice would be $h f(a+h)$ and continue on until the last slice $h f(b)$ : this time, the lower limit is not included and upper limit is. This is called the *right rectangle method* or the *right Riemann sum*.

For the analog of central differences, we can use rectangular slices with same width $h$ but height evaluated halfway between the $x$ points. This approach is called the *mid-point Riemann sum* or simply the *mid-point* method. This essentially ends up summing from $h f(a+h/2)$ all the way until $h f(b-h/2)$. 

Let's compare the three methods:

In [24]:
h=0.1

# forward rectangles or left point method: include a, exclude b
x_fwd=np.arange(a,b,h)
# backward rectangles or right point method: exclude a, include b
x_bwd=np.arange(a+h,b+h,h)
# central rectangles: exclude a and b, include all mid-points
x_cen=np.arange(a+h/2,b,h)

# perform sums in each case
Y_fwd=sum(h*y(x_fwd))
Y_bwd=sum(h*y(x_bwd))
Y_cen=sum(h*y(x_cen))

print('Forward rectangles with a gap size of %.4f is %.4f' %(h,Y_fwd))
print('Backward rectangles with a gap size of %.4f is %.4f' %(h,Y_bwd))
print('Central rectangles with a gap size of %.4f is %.4f' %(h,Y_cen))

Forward rectangles with a gap size of 0.1000 is 57.2300
Backward rectangles with a gap size of 0.1000 is 62.8300
Central rectangles with a gap size of 0.1000 is 59.9850


Like with finite differences, the central (or mid-point) rectangles give a more accurate result for a fixed gap size. Reducing the gap size improves all methods, but the mid-point method is always more accurate.




<a id='taylor'></a>



### Taylor series expansions and truncation errors

Like before, Taylor series approximations can quantify errors associated with numerical integration methods. We can then write out the Taylor series expansion of the value of the function at any arbitrary point $x$ around the left point $x_i$ as:

$$  f(x) = f(x_i) + (x-x_i) f'(x_{i}) + \frac{(x-x_i)^2}{2!} f''(x_{i}) +  \frac{(x-x_i)^3}{3!} f'''(x_{i}) + \ldots $$

Which means that the integral is 
$$\int_{x_i}^{x_{i}+h} f(x) \,dx  = \int_{x_i}^{x_{i}+h} f(x_i)\,dx+ \int_{x_i}^{x_{i}+h} (x-x_i) f'(x_{i})\,dx+\int_{x_i}^{x_{i}+h} \frac{(x-x_i)^2}{2!} f''(x_{i})\,dx+ \ldots $$

Each integrand contains values of functions (or derivatives) evaluated at a point and are therefore constant. Pulling those out of the integral and simplifying gives:

$$\int_{x_i}^{x_{i}+h} f(x) \,dx  = h f(x_i) + \frac{h^2}{2} f'(x_{i}) + \frac{h^3}{6} f''(x_{i})\,dx+ \ldots $$

If $h$ is small, we can truncate after the first term to get

$$\int_{x_i}^{x_{i}+h} f(x) \,dx  = h f(x_i) + O(h^2) $$

The forward or 'left' rectangle method is therefore second-order accurate in each interval. However, remember that an integral is the *sum* of multiple such intervals. Smaller $h$ also means that there are more intervals to sum over since there are a total of $(b-a)/h$ number of intervals. So, to get a sense of the total error, we have to multiply the indidual interval error by $(b-a)/h$, which tells us that the method is $O(h)$ or first-order accurate when summed over the entire integration interval.

Try the same math with the backward or 'right' rectangle method. What do you expect the order of accuracy to be?


For the mid-point method, writing out the Taylor series expansion of the value of the function at any arbitrary point $x$ around the mid-point $x_i+h/2$ gives:

$$  f(x) = f(x_i+h/2) + (x-x_i-h/2) f'(x_{i}) + \frac{(x-x_i-h/2)^2}{2!} f''(x_{i}+h/2) +  \frac{(x-x_i-h/2)^3}{3!} f'''(x_{i}+h/2) + \ldots $$

Which means that the integral in one interval is 

$$\int_{x_i}^{x_{i}+h} f(x) \,dx  = \int_{x_i}^{x_{i}+h} f(x_i+h/2)\,dx+ \int_{x_i}^{x_{i}+h} (x-x_i-h/2) f'(x_{i}+h/2)\,dx+\int_{x_i}^{x_{i}+h} \frac{(x-x_i-h/2)^2}{2!} f''(x_{i}+h/2)\,dx+ \ldots $$

Simplifying term by term gives (do this integration by hand, convince yourself that the coefficients are correct):

$$\int_{x_i}^{x_{i}+h} f(x) \,dx  = h f(x_i+h/2) +\frac{h^3}{24} f''(x_{i}+h/2)+ \ldots $$

Importantly, the term proportional to $h^2$ cancels out because the limits are on either side of and equidistant from the mid-point. This means that truncating after the first term gives

$$\int_{x_i}^{x_{i}+h} f(x) \,dx  = h f(x_i+h/2) + O(h^3) $$

The mid-point rectangle method is therefore *third-order* accurate in each interval. Following the same logic as above, and recognizing that there are $O(1/h)$ such intervals over which errors add up, we can say that the total integral is *secon-order* accurate. Like with finite differences, the central differences therefore gives a better accuracy for a given choice of gap size.

The following code takes any user-defined function and integration limits to give the definite integral to *global* second-order accuracy in grid size (which is set to a thousandth of the difference between limits, and can be changed as needed or even taken as an input to the function ...):

In [31]:
def func_integ(f,a,b):
    h=(b-a)/1000
    
    # generate array midpoints
    x_cen=np.arange(a+h/2,b,h)
    
    # integral as weighted sum
    Y_cen=sum(h*y(x_cen))
    
    return Y_cen

In [35]:
func_integ(y,2,4)

59.99999399999255

<a id='trapz'></a>



### Trapezoidal rule


The idea of the mid-point method can be directl extended to discrete sets of data. Since we do not have the functional form for a given/measured/simulated data set, the functional value at each mid-point is estimated as the average of the functional values at the edges of each interval. So, the approximate area of each such sliver is 

$$\int_{x_i}^{x_{i}+h} f(x) \,dx  \approx \frac{h \left[ f(x_i)+f(x_{i+1}) \right]}{2} $$

This is called the **trapezoidal rule** because it approximates each interval as a trapezoid, so that the area of each small sliver is the width times the average of the unequal sides. Written this way, it is also obvious that the trapezoidal rule is simply the average of the left-rectangle and right-rectangle methods.

This method can be easily extended to find the integral of a finite data set $(x_i,y_i)$ constaining $n$ points (so that $i=0,1,2,\ldots,n-1$) is:

$$ I  = \frac{(y_0+y_1)h}{2}+\frac{(y_1+y_2)h}{2}+\ldots+\frac{(y_{n-3}+y_{n-2})h}{2} + \frac{(y_{n-2}+y_{n-1})h}{2}   = \sum_{i=0}^{n-2}\frac{(y_i+y_{i+1})h}{2}  $$

 


Another way to write the trapezoidal method is 

$$  I= \frac{y_0h}{2}+\frac{y_{n-1}h}{2} + \sum_{i=1}^{n-2} y_ih $$

In other words, each point is weighted by $h$, except the end points that get a weight of $h/2$ each. This can be easily coded up:

In [1]:
def trapz(y,x):
    # determine spacing array
    h=x[1]-x[0]
    
    # sum of h x non-end points
    I=sum(h*y[1:-1])
    
    # add one-half h x end points
    I+=(y[0]+y[-1])*h/2
    
    return I

Let's check with the velocities of the space shuttle from tutorial 7:

In [67]:
t=np.arange(0,121,10)
v=np.array([-12.8, 62.6 , 131.55, 206.65, 262.9 , 312.  , 362.5 , 412.75, 511.4 , 577.  , 635.05, 665.7, 617.7])

trapz(v,t)

44425.5

Compare how well this approximates the height data from tutorial 7. For intermediate heights, integrate only upto a certain time: for example, the `data_integ(v[:5],t[:5])` gives integrated height up to $t=40$ seconds.

An error analysis of the trapezoidal method can be performed by writing the Taylor expansion for each interval with respect to *both* edges $x_i$ and $x_{i+1}$, and finding the term by term average (add the two line and divide by 2), and then integrating every term like before. Again, in this sense, the trapezoidal method is the average of the left and right rectangle rules. This calculation is a bit involved (try it if you are adventurous or curious) and works out to be:

$$\int_{x_i}^{x_{i}+h} f(x) \,dx  = \frac{h \left[ f(x_i)+f(x_{i+1}) \right]}{2} -\frac{h^3}{12} f''(x_{i})+ \ldots $$

Therefore, the trapezoidal rule is locally accurate to third order in each sliver (and therefore accurate to second order across the entire integration interval) but with an error of the opposite sign as the mid-point rule. The trapezoidal rule slightly overpredicts the value of the integral while the midpoint rule slightly underpredicts it.

Finally, an equivalent way to derive the trapezoidal rule is to approximate the function between each $x_i$ and $x_{i+1}$ with a first-order Lagrange interpolant: i.e., a straight line. Such a line would take the form (see tutorial 4):

$$ f(x)= \frac{x-x_{i+1}}{x_i-x_{i+1}} f(x_i) +\frac{x-x_i}{x_{i+1}-x_i} f(x_{i+1}) = -\frac{(x-x_{i+1})}{h} f(x_i) +\frac{(x-x_i)}{h}f(x_{i+1}) $$

Integrating this interpolant in each sliver acorss $x$ (between $x_i$ and $x_{i+1}$) gives the trapeoidal rule.



<a id='simpson'></a>



### Simpson's rule

The trapezoidal rule is the simplest of so-called **Newton-Cotes formulas** which estimate the integral of a given data set by weighting every $y$ point with a value and summing them all. In the trapezoidal rule, the weights were $h/2$ at end points and $h$ for every other point.

A natural improvement to the trapezoidal rule is to use higher-order polynomial interpolants between data points. The simplest improvement then is a quadratic polynomial (or a parabola) that passes through sets of three points (remember how a polynomial of degree n can pass uniquely through n+1 points). An integration method built on this idea is called **Simpson's rule** (or Simpson's 1/3rd rule for reasons we will see below). 

The quadratic Lagrange interpolation polynomial across three data points $x_{i-1}$, $x_{i}$ and $x_{i+1}$ is (see tutorial 4):

$$ f(x)= \frac{(x-x_{i})(x-x_{i+1})}{(x_{i-1}-x_{i})(x_{i-1}-x_{i+1})} f(x_{i-1}) + \frac{(x-x_{i-1})(x-x_{i+1})}{(x_{i}-x_{i-1})(x_{i}-x_{i+1})} f(x_{i}) + \frac{(x-x_{i-1})(x-x_{i})}{(x_{i+1}-x_{i-1})(x_{i+1}-x_{i})} f(x_{i+1}) \\ 
~\\
=\frac{(x-x_{i})(x-x_{i+1})}{2h^2} f(x_{i-1}) - \frac{(x-x_{i-1})(x-x_{i+1})}{h^2} f(x_{i}) + \frac{(x-x_{i-1})(x-x_{i})}{2h^2} f(x_{i+1})$$


Integrating this across the two-sliver set (or across the range of the three data points $x_{i-1}$, $x_{i}$ and $x_{i+1}$ gives 

$$\int_{x_i-h}^{x_{i}+h} f(x) \,dx  = \frac{hf(x_{i-1})}{3} + \frac{4hf(x_{i})}{3} + \frac{hf(x_{i+1})}{3} $$

To evaluate the integral across the enire range $[a,b]$, then, we would simply move to the next set of three points and evaluate the sum of the two-sliver set, and so on. Clearly, this method therefore would work **only with an odd number of data points**. With a user-defined function, this is not a severe restriction as we can create any number of points in between the limits $a$ and $b$. With a finite data set, however, we can only use Simpson's rule in specific cases (or we need to forfeit an point to make it an odd data set).

In a range $[a,b]$ split into $n$ points, the Simpson's rule corresponds to sequentially multipying every 3-element subset with $[h/3,4h/3,h/3]$. Across the entire range of $(x_i,y_i)$ points, this gives:

$$ I  = \frac{(y_0+4y_1 + y_2)h}{3}+\frac{(y_2+4y_3+ y_4)h}{3}+\ldots+ \frac{(y_{n-3}+4y_{n-2}+y_{n-1})h}{3}$$

with the necessary condition that $n$ is odd (or $n-1$ is even). Another way to write the Simpson's 1/3rd rule is therefore:

$$  I= \frac{y_0h}{3}+\frac{y_{n-1}h}{3} + \sum_{i=2, ~{\rm even}~ i}^{n-3} \frac{2 y_i h}{3}+ \sum_{i=1, ~{\rm odd}~ i}^{n-2} \frac{4 y_i h}{3}. $$

In [2]:
# simpson's rule for data points

def simps(y,x):
    
    if len(y) % 2 == 0:
        print("provide odd number of data points")
        return
    
    # determine spacing array
    h=x[1]-x[0]
    
    # end points
    I=(y[0]+y[-1])*h/3
    
    # even points, start at 2, move in steps of 2 until n-3
    I+=sum(y[2:-1:2])*2*h/3
    
    # odd points, start at 1, move in steps of 2 until n-2
    I+=sum(y[1:-1:2])*4*h/3
    
    return I

Despite the restriction of odd number of data sets, Simpson's rule is extremely popular and powerful because of it's accuracy. A formal error analysis is mathematically involved, but an easy way to see the error is to write the mid-point rule and trapezoidal rule from above between $x_{i-1}$ and $x_{i+1}$ with a step size of $2h$:

$$I_{\rm midpoint}: \qquad  \int_{x_{i}-h}^{x_{i}+h} f(x) \,dx  = 2h f(x_i) +\frac{h^3}{3} f''(x_{i})+ O(h^5) $$

$$ I_{\rm trapezoidal}: \qquad \int_{x_i -h }^{x_{i}+h} f(x) \,dx  =  h \left[ f(x_{i-1})+f(x_{i+1}) \right] -\frac{2h^3}{3} f''(x_{i})+ O(h^5) $$

Adding twice the first line to the second line eliminates the $O(h^3)$ term; dividing the result by three gives the Simpson's rule with an estimate for the error:

$$\frac{2I_{\rm midpoint}+I_{\rm trapezoidal}}{3}: \qquad \int_{x_{i}-h}^{x_{i}+h} f(x) \,dx = \frac{hf(x_{i-1})}{3} + \frac{4hf(x_{i})}{3} + \frac{hf(x_{i+1})}{3} + O(h^5), $$

which is Simpson's rule, and which is, therefore, fifth-order accurate within this range. Simpson's rule is thus *fourth-order accurate across the entire integration range $[a,b]$*. 

More accurate Newton-Cotes formulas can be obtained by using higher-order polynomials to locally interpolate across more sets of points. For example, a cubic Lagrange interpolant gives the so-called Simpson's "3/8th rule" (as against the "1/3rd rule" above). See practice problem 4 for details.

<a id='more'></a>



## (8.2) More integration strategies


Once we know the tricks above, more complicated or unusual integration can often be simplified.

<a id='multi'></a>



### Multidimensional integrals

We can easily extend ideas of numerical integration to double or triple integrals. Much like with partial derivative in the last tutorial, the idea is to hold one dimension constant and perform the same calculation as above along the other.

For example, the integral of a function $f(x,y)$ over a 2D area is represented in Cartersian co-ordinates as:

$$ I = \int_{A} f(x,y)\,dA = \int_{x=a}^{b} \left[ \int_{y=c}^{d} f(x,y)\,dy \right]\,dx $$

Written this way, it is obvious that the area integral can be done sequentially as two line integrals:

$$ I = \int_{A} f(x,y)\,dA = \int_{x=a}^{b} G(x)\,dx, \qquad {\rm where} \qquad G(x)= \int_{y=c}^{d} f(x,y)\,dy $$

To do this numerically, we would first split the $x$ and $y$ directions into $m$ and $n$ points with spacing $h_x$ and $h_y$ respectively. Using the trapezoid rule as an example, the inner integral at $x_i$ then becomes:

$$ G(x_i) = \frac{h_y f(x_i,c)}{2}+\frac{h_y f(x_i,d)}{2} + \sum_{j=1}^{n-2} h_y f(x_i,y_j) $$

Note that the location of the $x$ variable does not change in this step. We are holding it constant and integrating along $y$ every time. Once $G(x_i)$ is known at every $x_i$, the total integral can be performed the same way:

$$ I = \frac{h_x G(a)}{2}+\frac{h_x G(b)}{2} + \sum_{i=1}^{n-2} h_x G(x_i) $$

Let's try to find the area of the hat-shaped function from tutorial 2:

$$ \int_{A} \sin\left(\sqrt{x^2+y^2}\right) \,dA, $$

where the area $A$ corresponds to the region $x\in[-1,1]$ and $y\in [-1,1]$. This region is a square and we are free to choose $h_x$ and $h_y$ (smaller the better). The following code implements the trapezoidal rule in 2D:

In [19]:
def f(x,y): return np.sin(np.sqrt(x**2+y**2))

# choose spacing, need not be the same
hx=0.01
hy=0.01

# generate x and y
x=np.arange(-1,1+hx,hx)
y=np.arange(-1,1+hy,hy)

# trapezoid rule along y
G=np.zeros(x.shape)

for i in range(len(x)):
    G[i]= hy*sum(f(x[i],y[1:-1])) + hy*f(x[i],y[0])/2 + hy*f(x[i],y[-1])/2

# integrate G along x
I = hx*sum(G[1:-1]) + hx*G[0]/2 + hx*G[-1]/2

print(I)

2.663523910358142


The same can be done with Simpsons's rule as well. Try it. Make sure to use an odd number of points for Simpson's rule to work.

In fact, the integration limits of the inner integral need not even be constants! The trick above work even for integration limits that are functions of $x$. In other words, the area of integration does not have to be a square and can be any arbitrary shape with limits $c(x)$ and $d(x)$ in the y-direction. In this case, the $y$ bounds and spacing at each $x$ location is different, and therefore has to be done within the loop. The following code calculates the integral of the same function in the area contained between $x=-1$ and $x=1$ and $y=-1-x^2$ and $y=1+x^2$. There are many ways to choose the spacing in $y$; we could keep it constant or change it with $x$. Here we choose is such that there are always 100 points, so it changes as each $x$ location.

In [20]:
# choose spacing, need not be the same
hx=0.01

# generate x
x=np.arange(-1,1+hx,hx)

# trapezoid rule along y
G=np.zeros(x.shape)

for i in range(len(x)):
    # generate new y for each x location
    y=np.linspace(-1-x[i]**2,1+x[i]**2,100)
    # determine hy now
    hy=y[1]-y[0]
    G[i]= hy*sum(f(x[i],y[1:-1])) + hy*f(x[i],y[0])/2 + hy*f(x[i],y[-1])/2

# integrate G along x
I = hx*sum(G[1:-1]) + hx*G[0]/2 + hx*G[-1]/2

print(I)

3.9496156597602727


<a id='inbuilt'></a>



### Inbuilt python routines

Python has inbuilt routines to do integrals too. The function `scipy.integrate.trapz` is identical to our trapezoidal rule above. And `scipy.integrate.simps` is a more sophisticated implementation of the Simpson's rule (with a control for even/odd number of points etc). Read about the details in the official manual for the [Simpson's rule](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.simps.html). In terms of usage, they work as follows:

In [77]:
t=np.arange(0,121,10)
v=np.array([-12.8, 62.6 , 131.55, 206.65, 262.9 , 312.  , 362.5 , 412.75, 511.4 , 577.  , 635.05, 665.7, 617.7])

trapz(v,t)

print('Our own trapezoidal method gives: %.6f' %trapz(v,t))
print('Our own Simpson rule gives: %.6f' %simps(v,t))

import scipy.integrate as si

print('\nScipy inbuilt trapezoidal method gives: %.6f' %si.trapz(v,t))
print('Scipy inbuilt Simpson rule gives: %.6f' %si.simps(v,t))

Our own trapezoidal method gives: 44425.500000
Our own Simpson rule gives: 44528.333333

Scipy inbuilt trapezoidal method gives: 44425.500000
Scipy inbuilt Simpson rule gives: 44528.333333


We can see that our methods give identical results as the `scipy` inbuilt functions. 


<a id='exer'></a>



### Practice Problems

(1) Evaluate using *both* trapezoidal and Simpson's 1/3rd rule:

(1a) The following definite integral with an exact value of $\pi/2$ (compare your solutions to this)

$$\int_0^\pi \sin^2(x)\,dx $$

(1b) The following definite integral with an exact value of $\pi$ (compare your solutions to this) 

$$\int_{-1}^{1} \frac{2}{1+x^2}\,dx $$

(1c) Evalute this integral by hand and compare your numerical results:

$$\int_{-3}^{3} \left( 1-\frac{x^2}{9} \right) \,dx $$

(2) Wine barrels are circular in cross-section with a radius $r(z)$ that changes with height. Estimating the volume $V$ and surface area $S$ of such a barrel then involves adding up these circular slices. In other words,

$$ S = 2\pi \int_0^H r\,dz \qquad {\rm and }\qquad V = \pi \int_0^H r^2\,dz $$

where $H$ is the total height, and $z$ is measured from the floor up. Use the following measurements to approximate both $V$ and $S$, where $d$ is the radius in excess of a cylindrical shape of constant radius $9.6$ inches.

$$\begin{matrix} 
{\rm z~(inches)} & 0 & 6 &12 & 18 & 24 & 30 & 36 \\
{\rm d~(inches)} & 0 & 1.25 & 2 & 2.25 & 2 & 1.25 & 0\\
        \end{matrix}$$
        
These are number for a 60 gallon barrel. Find the conversion between cubic inches and gallons, and see if you answer is reasonable.

(3) The radiative heat loss from an ideal object is well approximated by Stefan-Boltzmann law:

$$ j = \sigma T^4, $$

where $j$ is the thermal energy lost in ${\rm W/m^2}$, $T$ is the absolute temperature, and $\sigma = 5.67 \times 10^{-8}\, {\rm W/m^2/K^{-4}}$ is Stefan's constant. From our temperature data in `smoothtemp.txt`, find $j$ at every point on the plate. Recall that the plate is 20 cm by 10 cm: find the total heat energy lost by radiation by integrating $j$ across this entire area.

(4) Simpson's 3/8th rule is obtained by interpolating sets of 4 data points (or three 'panels') by cubic polynomials. The formula thus obtained is:

$$ I  = \frac{(y_0+3y_1 + 3y_2 + y_3)3h}{8}+\frac{(y_3+3y_4+ 3y_5+y_6)3h}{8}+\ldots+ \frac{(y_{n-4}+3y_{n-3}+3y_{n-2}+y_{n-1})3h}{8}$$

The necessary condition is then that the number of panels is divisible by 3. Equivalently, the number of data points should be one over a multiple of 3.

Write a function to compute an integral using this formula, given $(x_i,y_i)$ data points. Use this to evaluate the integrals in problem 1.

(5) The arclength of a curve $f(x)$ between $a$ and $b$ is given by 

$$ L = \int_a^b \sqrt{1+\left[ f'(x) \right]^2} \, dx $$

(5a) Use the trapezoidal integration scheme to find the arclength of $\sin(x)$ between $0$ and $\pi$.

(5b) The central part of the Golden Gate bridge is suspended from two towers that rise 500 feet above the road and are 4200 feet apart. The height of the central cables measured every 300 feet along the bridge are given below:

$$\begin{matrix} 
{\rm x~(feet)} & 0 & 300 & 600 & 900 & 1200 & 1500 & 1800 & 2100 & 2400 & 2700 & 3000 & 3300 & 3600 & 3900 & 4200 \\
{\rm h~(feet)} & 500 & 365 & 250 & 160 & 90 & 40 & 10 &0 & 10 & 40 & 90 & 160 & 250 & 365 & 500\\
        \end{matrix}$$
        
First, plot this data.

Our objective is to find the length of the central suspension cables. To use the formula above, we need the derivative. Use the second-order accurate `data_deriv` function we developed in tutorial 7 to find $f'(x)$ at every point:

Use `trapz` to integrate and find the arc length of the central suspension cable:

(6) Evaluate in the domain $-\pi\leq x \leq \pi$ and $-\pi\leq y \leq \pi$

$$ \int_{A} \sin^2(x+y)\cos^2(x-y)\, dA$$

(6b) Evaluate the integral in the domain $A$ corresponding to the unit circle of radius 1. 

*Hint: this is easier in polar coordinates: convert $x$ and $y$ to $r$ and $\theta$ ...*